In [98]:
!pip install pyvis

In [99]:
# wrong_list = ['5a44c7da', 'd9346748', 'a01b13cd', '32159b42', 'df1aaea1', 'a33c6fe5', '656029da', '7a5d46ab']

In [ ]:
GEN0_OF_INTEREST_FACES = [
    '28413638', '5f80888c', '32159b42', 
    'd9346748', 'df1aaea1', '656029da', 
    '5a44c7da', 'a01b13cd', 'a33c6fe5', '7a5d46ab'
]
# GEN1_OF_INTEREST_FACES = None
GEN1_OF_INTEREST_FACES = [
    '71157398',  'f94ec936', '8ebe8b02',
    'ab83bbf7',   '4a2e5720', 'adfd099e', 
    '9576e2df',  '1b81e25d',  'e54cbcce',
    '6e8de1d9', 'e52b237e', 'd53a9a9c',  
]

# '60a3b493','24197935', '8a112cf6', 'ef8dfcb7','d10dd733', '9caa46ea', 'affaaf6a','ef0c74a8','6940bd30',
# 'b89b83c3', 'fbf5c37d', 'cada7d68', '3281f769','79836dc0', 'cc11aad2','74001433', 



In [101]:
# [ind for ind in OF_INTEREST_FACES if ind not in wrong_list]

In [102]:
import pandas as pd
import math
from pyvis.network import Network

# Load your CSV
df = pd.read_csv("INDIVIDUALS_EVOLVING_EMOTE_BIG_RUYN/all_individuals_evaluation_log.csv")

net = Network(height="900px", width="100%", directed=True, notebook=False)

# Disable physics completely
net.toggle_physics(False)

# Limit to top 100 by fitness per generation
top_nodes = []
for gen, group in df.groupby("generation"):
    top = group.nlargest(100, "fitness")
    top_nodes.append(top)
top_df = pd.concat(top_nodes)

# --- Add INITIAL node ---
initial_img = "INITIAL.png"
initial_label = "INITIAL"
initial_scores = {
    "angry": 0.47, "disgust": 0.0, "fear": 0.99, "happy": 0.04,
    "sad": 1.22, "surprise": 0.30, "neutral": 96.95
}
initial_html = f"""
<div style="text-align:center; font-size:12px; user-select:text;">
    <img src="{initial_img}" width="120"><br>
    <b>{initial_label}</b><br>
    Angry: {initial_scores['angry']:.2f} | Disgust: {initial_scores['disgust']:.2f}<br>
    Fear: {initial_scores['fear']:.2f} | Happy: {initial_scores['happy']:.2f}<br>
    Sad: {initial_scores['sad']:.2f} | Surprise: {initial_scores['surprise']:.2f}<br>
    Neutral: {initial_scores['neutral']:.2f}
</div>
"""
net.add_node(
    "INITIAL",
    title=initial_html,
    label="",  # leave empty so no canvas label
    shape="image",
    image=initial_img,
    x=0, y=0, fixed=False
)

# --- Add face nodes with manual positioning ---
gen_x_gap = 300   # horizontal spacing between generations
gen_y_gap = 75    # vertical spacing between nodes

count_per_gen = [0, 0, 0,0,0]

for gen, group in top_df.groupby("generation"):
    nodes = group.to_dict("records")
    n = len(nodes)
    # grid_size = math.ceil(math.sqrt(n))  
    
    for i, row in enumerate(nodes):
        indiv_id = row["individual_id"]
        label = f"output_gen{gen}_{indiv_id[:8]}"
        if gen == 0 and GEN0_OF_INTEREST_FACES is not None and indiv_id[:8] not in GEN0_OF_INTEREST_FACES:
              continue
        if gen == 1 and GEN1_OF_INTEREST_FACES is not None and indiv_id[:8] not in GEN1_OF_INTEREST_FACES:
              continue
        # else:
        #     print(f'processing {label}')

        img = row["face_pic"]

        node_html = f"""
        <div style="text-align:center; font-size:12px; user-select:text;">
            <img src="{img}" width="120"><br>
            <b>{label}</b><br>
            Angry: {row['angry']:.2f} | Disgust: {row['disgust']:.2f}<br>
            Fear: {row['fear']:.2f} | Happy: {row['happy']:.2f}<br>
            Sad: {row['sad']:.2f} | Surprise: {row['surprise']:.2f}<br>
            Neutral: {row['neutral']:.2f}
        </div>
        """
        # grid position
        row_i = count_per_gen[gen]
        count_per_gen[gen] = count_per_gen[gen] + 1
        col_i = gen + 1
        if 'INITIAL' in img:
            col_i = 0
        x = gen * gen_x_gap + (col_i) * gen_x_gap / 2
        y = (row_i) * gen_y_gap
        
        net.add_node(
            indiv_id if indiv_id != '28413638' else '284136g8',
            title=node_html,
            label="",   # no canvas text
            shape="image",
            image=img,
            x=x,
            y=y,
            fixed=False
        )

# --- Add edges ---
existing_nodes = set(net.get_nodes())

valid_nodes = set(top_df["individual_id"]) | {"INITIAL"}
# if GEN0_OF_INTEREST_FACES is None:
for _, row in top_df.iterrows():
    # print([row["parent1_individual_id"], row["parent2_individual_id"]])
    # for parent in [row["parent1_individual_id"], row["parent2_individual_id"]]:
      if row["individual_id"] in existing_nodes:
        # if parent in existing_nodes:
        print('adding edge from ', row["parent1_individual_id"], row["individual_id"])
        net.add_edge(row["parent1_individual_id"], row["individual_id"])
        # if parent in existing_nodes:
        print('adding edge from ', row["parent2_individual_id"], row["individual_id"])
        net.add_edge(row["parent2_individual_id"], row["individual_id"])

# --- Options ---
net.set_options("""
{
  "edges": {
    "smooth": {
      "type": "curvedCW",
      "roundness": 0.2
    }
  },
  "nodes": {
    "borderWidth": 2,
    "shadow": true
  },
  "interaction": {
    "dragNodes": true,
    "zoomView": true,
    "tooltipDelay": 50
  },
  "physics": {
    "enabled": false
  }
}
""")

net.write_html("faces_lineage.html", notebook=False)


adding edge from  INITIAL 284136g8
adding edge from  INITIAL 284136g8
adding edge from  INITIAL 32159b42
adding edge from  INITIAL 32159b42
adding edge from  INITIAL 656029da
adding edge from  INITIAL 656029da
adding edge from  INITIAL a33c6fe5
adding edge from  INITIAL a33c6fe5
adding edge from  INITIAL 5a44c7da
adding edge from  INITIAL 5a44c7da
adding edge from  INITIAL a01b13cd
adding edge from  INITIAL a01b13cd
adding edge from  INITIAL 5f80888c
adding edge from  INITIAL 5f80888c
adding edge from  INITIAL d9346748
adding edge from  INITIAL d9346748
adding edge from  INITIAL df1aaea1
adding edge from  INITIAL df1aaea1
adding edge from  INITIAL 7a5d46ab
adding edge from  INITIAL 7a5d46ab
adding edge from  284136g8 ab83bbf7
adding edge from  df1aaea1 ab83bbf7
adding edge from  284136g8 8ebe8b02
adding edge from  d9346748 8ebe8b02
adding edge from  284136g8 9576e2df
adding edge from  a33c6fe5 9576e2df
adding edge from  7a5d46ab 71157398
adding edge from  df1aaea1 71157398
adding edge 